In [9]:
!pip install --quiet datasets pythainlp transformers sentencepiece protobuf peft

## Get Data, Process, and Tokenize

In [2]:
from datasets import load_dataset
from pythainlp.tokenize import subword_tokenize
import pandas as pd
import re

In [3]:
dataset = load_dataset("Bhunakit/paraphrasethai")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
data = pd.DataFrame({'original': dataset['train']['original'], 'paraphrased': dataset['train']['paraphrased']})

In [7]:
data.head()

,original,paraphrased
0,รอกันมา 2 สัปดาห์ บอลสโมสรลีกต่างๆของยุโรปก็กล...,เมื่อผ่านไป 2 สัปดาห์แล้ว ฟุตบอลสโมสรในลีกต่าง...
1,รองผู้ว่าราชการจังหวัดภูเก็ตลงพื้นที่ตรวจสอบเก...,พระองค์ในฐานะผู้ว่าราชการจังหวัดภูเก็ตจะออกไปต...
2,พาณิชย์ ยืนเป้าส่งออกปีนี้ยังเติบโต 5% ไว้เป็น...,พาณิชย์เป้าหยุดเวลาปีนี้เพิ่มขึ้น 5% เป็นกำลัง...
3,ชาวบ้านหมู่ 1 ต.บ้านกลาง อ.อ่าวลึก จ.กระบี่ ออ...,ชาวบ้านในหมู่บ้านที่ 1 ที่ตั้งอยู่ในตำบลบ้านกล...
4,ยายวัย 62 ปีเหยื่อรถแรลลี่ข้ามจังหวัดถูกรถชนกร...,ยายอายุ 62 ปีผู้เสียชีวิตในอุบัติเหตุการชนรถแร...


## Defining Architecture

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForSequenceClassification, LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
class TransformerGenerator(torch.nn.Module):
    def __init__(self):
        super(TransformerGenerator, self).__init__()
        # Load the pre-trained GPT model for causal language modeling in paraphrasing task
        self.tokenizer = LlamaTokenizer.from_pretrained("openthaigpt/openthaigpt-1.0.0-beta-7b-chat-ckpt-hf", torch_dtype=torch.float16)
        self.model = LlamaForCausalLM.from_pretrained("openthaigpt/openthaigpt-1.0.0-beta-7b-chat-ckpt-hf", torch_dtype=torch.float16).to(device)
        self.model = PeftModel.from_pretrained(self.model, "Bhunakit/openthaigpt_paraphrase")

        self.model.config.pad_token_id = self.tokenizer.pad_token_id = 0  # unk
        self.model.config.bos_token_id = 1
        self.model.config.eos_token_id = 2
        self.model.half()
        
        self.model.to(device)
        self.model.eval()
        
        self.model = torch.compile(self.model)

    def forward(self, input_text, max_length=200):
        # Tokenize input text and convert to tensor
        input_ids = self.tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
        input_ids = input_ids['input_ids'].to(device)
    
        # Generate paraphrased text batch
        with torch.no_grad():
            # Adjust the generation length if needed
            gen_max_length = max_length + 20
            generated_ids = self.model.model.generate(input_ids, max_length=gen_max_length)
    
        # Decode the generated ids to text batch
        paraphrased_texts = [self.tokenizer.decode(g_id, skip_special_tokens=True) for g_id in generated_ids]

        paraphrased_texts = [re.sub(r'\[.*?\]', '', re.sub(r'\n', '', text)) for text in paraphrased_texts]
        paraphrased_texts = [text.strip() for text in paraphrased_texts]
    
        return paraphrased_texts

In [13]:
class TransformerDiscriminator(torch.nn.Module):
    def __init__(self):
        super(TransformerDiscriminator, self).__init__()
        # Load the pre-trained BERT model for sequence classification
        self.model = AutoModelForSequenceClassification.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", num_labels=2).to(device)
        self.tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased", padding_side='left')

        # Move model to GPU if available
        if torch.cuda.is_available():
            self.model = self.model.to('cuda')

    def forward(self, generated_text):
        # Tokenize the generated text
        encodings = self.tokenizer(generated_text, return_tensors="pt", padding=True, truncation=True, max_length=200).to(device)

        # Move input tensors to GPU if available
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']
        if torch.cuda.is_available():
            input_ids = input_ids.to('cuda')
            attention_mask = attention_mask.to('cuda')

        # Get model outputs
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        pred = torch.argmax(probs, dim=1)

        return probs


## Training

In [19]:
# Define hyperparameters
batch_size = 64
num_epochs = 3
learning_rate = 1e-5

In [15]:
generator = TransformerGenerator().to(device)
discriminator = TransformerDiscriminator().to(device)

# Optimizers
gen_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)
disc_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

In [17]:
class ParaphraseDataset(Dataset):
    def __init__(self, data):
        self.dataframe = data
        # Ensure the text columns are strings
        self.dataframe['original'] = self.dataframe['original'].astype(str)
        self.dataframe['paraphrased'] = self.dataframe['paraphrased'].astype(str)

    def __len__(self):
        # Return the length of the dataset
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Retrieve the original and paraphrased text at the given index
        original_text = self.dataframe.iloc[idx]['original']
        paraphrased_text = self.dataframe.iloc[idx]['paraphrased']
        return original_text, paraphrased_text

paraphrase_dataset = ParaphraseDataset(data)
 
train_dataloader = DataLoader(paraphrase_dataset, batch_size=batch_size, shuffle=True)

In [18]:
for epoch in range(num_epochs):
    for i, (original_real, paraphrased_real) in enumerate(train_dataloader):
        # Reset gradients
        gen_optimizer.zero_grad()
        disc_optimizer.zero_grad()

        # Generate paraphrases using generator
        paraphrased_fake = generator.forward(original_real)

        # Prepare labels: real (1) and fake (0)
        real_labels = torch.ones(len(original_real), dtype=torch.long, device=device)
        fake_labels = torch.zeros(len(paraphrased_real), dtype=torch.long, device=device)

        # Train Discriminator
        # Discriminator on real paraphrased data
        real_preds = discriminator.forward(paraphrased_real)
        # real_preds = torch.cat(real_preds, dim=0)
        real_loss = criterion(real_preds, real_labels)

        # Discriminator on fake paraphrased data
        fake_preds = discriminator.forward(paraphrased_fake)
        # fake_preds = torch.cat(fake_preds, dim=0)
        fake_loss = criterion(fake_preds, fake_labels)

        # Total loss and backprop
        disc_loss = real_loss + fake_loss
        disc_loss.backward()
        disc_optimizer.step()

        # Train Generator
        
        # Generate paraphrases
        paraphrased_fake = generator.forward(original_real)

        # Generator wants discriminator to mark the generated text as real
        gen_preds = discriminator.forward(paraphrased_fake)
        # gen_preds = torch.cat(gen_preds, dim=0)
        gen_loss = criterion(gen_preds, real_labels)

        # Backpropagation
        gen_loss.backward()
        gen_optimizer.step()

        # Logging the losses
        print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_dataloader)}], "
              f"Discriminator Loss: {disc_loss.item()}, Generator Loss: {gen_loss.item()}")

# Save models
torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Epoch [1/3], Step [1/31], Discriminator Loss: 1.3778178691864014, Generator Loss: 0.7677404284477234


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Epoch [1/3], Step [2/31], Discriminator Loss: 1.3341596126556396, Generator Loss: 0.8013405203819275


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

Epoch [3/3], Step [29/31], Discriminator Loss: 0.6741013526916504, Generator Loss: 1.2687307596206665


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Epoch [3/3], Step [30/31], Discriminator Loss: 0.6891337037086487, Generator Loss: 1.2801896333694458


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Epoch [3/3], Step [31/31], Discriminator Loss: 0.6676547527313232, Generator Loss: 1.2786674499511719


In [23]:
text = ['ทำไมท้องฟ้าวันนี้ถึงมีสีคราม', 'ทำไมวันนี้ท้องฟ้าสีฟ้าจัง']

gen_text = generator.forward(text)

[print(g, end='\n----------\n') for g in gen_text]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


ทำไมท้องฟ้าวันนี้ถึงเป็นสีฟ้า
----------
ทำไมวันนี้ท้องฟ้าสีฟ้าเช่นนี้
----------


[None, None]